## Peer-graded Assignment: Segmenting and Clustering Neighborhoods in Toronto

In [55]:
from bs4 import BeautifulSoup
import requests
import pandas as pd;

r  = requests.get("https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M")

soup = BeautifulSoup(r.text, "lxml")

#print(soup.prettify())

labels = ["Postcode","Borough","Neighbourhood"];
df_regions = pd.DataFrame(columns = labels)        


# Getting Raw data from wikipedia table 
for table in soup.find_all('table'):
    correctTable = False;

    for header in table.find_all('th'):
        if (header.text in labels):
            correctTable = True;
    if (not correctTable):
        break;
        

    for row in table.find_all('tr'):
        rowData = [];
        for cell in row.find_all('td'):
            rowData.append(cell.text.replace('\n',''));
        if (len(rowData) == len(labels)):
            df_regions.loc[len(df_regions)] = rowData;
        



In [56]:
df_regions.head()

,Postcode,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


In [112]:
# Do the cleaning
df_regions_clean = df_regions;

#Removes unassigned Boroughs
df_regions_clean = df_regions_clean[df_regions_clean.Borough != "Not assigned"]

# Joins doublicated Neighbourhoods together
df_regions_clean = df_regions_clean.groupby(["Postcode","Borough"]).agg({'Neighbourhood' : lambda x: ', '.join(x)})

# Resets the dataframe such that its usefull again
df_regions_clean = df_regions_clean.reset_index()

# Replaces 'not assigned' Neighbourhoods with the Borough
for indexNumber in df_regions_clean[df_regions_clean.Neighbourhood == "Not assigned"].index:
    df_regions_clean.iloc[indexNumber,:].Neighbourhood = df_regions_clean.iloc[indexNumber,:].Borough


In [114]:
#Some validating:

df_regions_clean[df_regions_clean.Borough == "Queen's Park"]



,Postcode,Borough,Neighbourhood
85,M7A,Queen's Park,Queen's Park


In [116]:
df_regions_clean.head()

,Postcode,Borough,Neighbourhood
0,M1B,Scarborough,"Rouge, Malvern"
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


In [117]:
# Shape of the regions?
print (df_regions_clean.shape)

(103, 3)
